In [ ]:
# ref: https://gist.github.com/amotl/8727ea2aa6e46c5b51a34c28b767d72c

In [ ]:
# docker run -it --rm --publish=5432:5432 --env=POSTGRES_HOST_AUTH_METHOD=trust postgres:13.2

In [1]:
import pandas as pd
import numpy as np
from itertools import cycle
def generate_fake_dataframe(size, cols, col_names = None, intervals = None, seed = None):
    
    categories_dict = {'animals': ['cow', 'rabbit', 'duck', 'shrimp', 'pig', 'goat', 'crab', 'deer', 'bee', 'sheep', 'fish', 'turkey', 'dove', 'chicken', 'horse'],
                       'names'  : ['James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth', 'Ahmed', 'Barbara', 'Richard', 'Susan', 'Salomon', 'Juan Luis'],
                       'cities' : ['Stockholm', 'Denver', 'Moscow', 'Marseille', 'Palermo', 'Tokyo', 'Lisbon', 'Oslo', 'Nairobi', 'Río de Janeiro', 'Berlin', 'Bogotá', 'Manila', 'Madrid', 'Milwaukee'],
                       'colors' : ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'black', 'white']
                      }
    default_intervals = {"i" : (0,10), "f" : (0,100), "c" : ("names", 5), "d" : ("2020-01-01","2020-12-31")}
    rng = np.random.default_rng(seed)

    first_c = default_intervals["c"][0]
    categories_names = cycle([first_c] + [c for c in categories_dict.keys() if c != first_c])
    default_intervals["c"] = (categories_names, default_intervals["c"][1])
    
    if isinstance(col_names,list):
        assert len(col_names) == len(cols), f"The fake DataFrame should have {len(cols)} columns but col_names is a list with {len(col_names)} elements"
    elif col_names is None:
        suffix = {"c" : "cat", "i" : "int", "f" : "float", "d" : "date"}
        col_names = [f"column_{str(i)}_{suffix.get(col)}" for i, col in enumerate(cols)]

    if isinstance(intervals,list):
        assert len(intervals) == len(cols), f"The fake DataFrame should have {len(cols)} columns but intervals is a list with {len(intervals)} elements"
    else:
        if isinstance(intervals,dict):
            assert len(set(intervals.keys()) - set(default_intervals.keys())) == 0, f"The intervals parameter has invalid keys"
            default_intervals.update(intervals)
        intervals = [default_intervals[col] for col in cols]
    df = pd.DataFrame()
    for col, col_name, interval in zip(cols, col_names, intervals):
        if interval is None:
            interval = default_intervals[col]
        assert (len(interval) == 2 and isinstance(interval, tuple)) or isinstance(interval, list), f"This interval {interval} is neither a tuple of two elements nor a list of strings."
        if col in ("i","f","d"):
            start, end = interval
        if col == "i":
            df[col_name] = rng.integers(start, end, size)
        elif col == "f":
            df[col_name] = rng.uniform(start, end, size)
        elif col == "c":
            if isinstance(interval, list):
                categories = np.array(interval)
            else:
                cat_family, length = interval
                if isinstance(cat_family, cycle):
                    cat_family = next(cat_family)
                assert cat_family in categories_dict.keys(), f"There are no samples for category '{cat_family}'. Consider passing a list of samples or use one of the available categories: {categories_dict.keys()}"
                categories = rng.choice(categories_dict[cat_family], length, replace = False, shuffle = True)
            df[col_name] = rng.choice(categories, size, shuffle = True)
        elif col == "d":
            df[col_name] = rng.choice(pd.date_range(start, end), size)
    return df

def pandas_to_sql(dburi):
    df.to_sql(
        con=dburi,
        name="testdrive",
        if_exists="replace",
        index=False,
        #method="multi",
        chunksize=5000,
    )

In [2]:
df = generate_fake_dataframe(
  size = 30, 
  cols = "cicffcd", 
  col_names = ["user", "age", "residence", "weight","height", "pet", "registered"],
  intervals = [("names",15), (18,25),("cities", 15), (73.2,95.0),
               (1.65,1.95), ("animals", 11), None],
  seed = None)

In [5]:
user_level_aggergated_data = df.groupby("user",as_index = False)[['weight','height']].mean()

In [10]:
payload_model = user_level_aggergated_data[['height','weight']].to_dict('list')
user_level_aggergated_data


,user,weight,height
0,Ahmed,85.442626,1.734614
1,Elizabeth,77.736308,1.721366
2,John,82.166965,1.773500
3,Juan Luis,78.923984,1.759042
4,Linda,75.653785,1.683153
5,Mary,87.851879,1.759215
6,Patricia,84.016987,1.862544
7,Robert,87.063977,1.713829
8,Salomon,86.749710,1.864065
9,Susan,87.547298,1.774000


In [19]:
from sklearn.linear_model import LinearRegression
# pip install scikit-learn
x = user_level_aggergated_data['height'].to_numpy().reshape(-1, 1)
y = user_level_aggergated_data['weight'].to_numpy().reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(x, y)
y_hat = linear_regressor.predict(x)
payload_model['y_hat'] = y_hat.tolist()

In [20]:
payload_model

{'height': [1.7346136988050496,
  1.7213656986891956,
  1.7734997999668396,
  1.7590424071538724,
  1.6831531826909703,
  1.759214969393133,
  1.8625437220369896,
  1.7138289341160977,
  1.864064529222078,
  1.7739999831375608,
  1.7464935670769848],
 'weight': [85.44262582147832,
  77.73630844100643,
  82.16696469449458,
  78.92398430617982,
  75.65378454414731,
  87.85187938291756,
  84.01698680385849,
  87.06397657276051,
  86.74970958687025,
  87.54729797992624,
  74.77548532922283],
 'response': [[81.48612353514223],
  [80.99287852708082],
  [82.93391755033632],
  [82.39564484711939],
  [79.57016320425743],
  [82.40206962537654],
  [86.24917039417885],
  [80.71227228688566],
  [86.30579256701316],
  [82.95254019903176],
  [81.92843072644025]],
 'y_hat': [[81.48612353514223],
  [80.99287852708082],
  [82.93391755033632],
  [82.39564484711939],
  [79.57016320425743],
  [82.40206962537654],
  [86.24917039417885],
  [80.71227228688566],
  [86.30579256701316],
  [82.95254019903176],
  

In [ ]:
# pandas_to_sql(dburi="postgresql://postgres@localhost/dwd")